In [6]:
import numpy as np
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
import joblib

In [7]:
model = joblib.load('./random_forest.joblib')

In [8]:
df = px.data.tips()

In [9]:
# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Classification Demo"),
        html.Label("Age"),
        dcc.Slider(
            id = 'age_slider',
            min = 0,
            max = 100,
            value = 30,
            step = 5,
            marks = {i: str(i) for i in range(10,101,10)},
    ),
    html.Label("Gender"),
    dcc.RadioItems(
    id = 'gender_picker',
    options=[
        {'label': 'Male', 'value': 'm'},
        {'label': 'Female', 'value': 'f'},
    ]),
    html.Label("Fare"),
            dcc.Slider(
            id = 'fare_slider',
            min = 0,
            max = 100,
            value = 30,
            step = 5,
            marks = {i: str(i) for i in range(10,101,10)},
    ),
    html.Label("Room Class"),
    dcc.RadioItems(
    id = 'class_picker',
    options=[
        {'label': '1', 'value': '1'},
        {'label': '2', 'value': '2'},
        {'label': '3', 'value': '3'},
    ]),
    html.Label("Title"),
    dcc.RadioItems(
    id = 'title_picker',
    options=[
        {'label': 'Mr.', 'value': 'Mr'},
        {'label': 'Mrs', 'value': 'Mrs'},
        {'label': 'Miss', 'value': 'Miss'},
        {'label': 'Master', 'value': 'Master'},
        {'label': 'Other', 'value': 'Other'},
    ]),
    dcc.Graph(id='graph'),
])
# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input('age_slider', 'value'),
    Input('gender_picker', 'value'),
    Input('fare_slider', 'value'),
    Input('class_picker', 'value'),
    Input('title_picker', 'value')]
)
def update_figure(age, gender, fare, class_val, title):
    
    #dont update if fields not filled in
    if gender == None or class_val == None or title == None:
        raise PreventUpdate
    else:
        values_dict = {'Pclass': 0, 'Age': 0, 'Fare': 0, 'male': 0, 'Miss': 0, 'Mr': 0, 'Mrs': 0, 'Other': 0}

        #update info
        values_dict['Pclass'] = class_val

        values_dict['Age'] = age

        values_dict['Fare'] = fare

        if gender == 'm':
            values_dict['male'] = 1.0

        if title != 'Master':
            values_dict[title] = 1

        sample_df = pd.DataFrame(values_dict, index = [0])
        
        vals = list(model.predict_proba(np.array(sample_df).reshape(1,-1))[0])
        
        graph_df = pd.DataFrame({'Categories':['Did not Survive' , 'Did Survive'], 'Probability':[vals[0], vals[1]]})

        return px.bar(graph_df, x='Categories', y='Probability', range_y = [-0.1,2])
        
        
    

In [10]:
# Run app and display result inline in the notebook
app.run_server(mode='inline')